<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/Teste%20Marco%20Passage%20%2B%20BM25%20%2B%20Rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/castorini/pygaggle.git

  Cloning https://github.com/castorini/pygaggle.git to /tmp/pip-req-build-yudo9soe
  Running command git clone -q https://github.com/castorini/pygaggle.git /tmp/pip-req-build-yudo9soe
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 9.1 MB 5.7 MB/s 
     |████████████████████████████████| 72.8 MB 80 kB/s 
     |████████████████████████████████| 22.3 MB 96.6 MB/s 
     |████████████████████████████████| 25.9 MB 69.1 MB/s 
     |████████████████████████████████| 12.8 MB 56.2 MB/s 
     |████████████████████████████████| 6.0 MB 24.3 MB/s 
     |████████████████████████████████| 454.3 MB 19 kB/s 
     |████████████████████████████████| 3.3 MB 54.7 MB/s 
     |████████████████████████████████| 72 kB 979 kB/s 
     |████████████████████████████████| 2.2 MB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 62.2 MB/s 
     |████████████████████████████████| 85 kB 5.6 MB/s 
     |

In [2]:
!pip install pyserini

In [3]:
from pyserini.search import SimpleSearcher

# `SimpleSearcher` defaults to BM25 scoring function.
searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

index-msmarco-passage-20201117-f87c94.tar.gz: 0.00B [00:00, ?B/s]

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [04:09, 8.88MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


In [4]:
import json
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.base import hits_to_texts

# Here's our query:
query = Query('are naturalization records public information')

hits = searcher.search(query.text, k=1000)
texts = hits_to_texts(hits)

print('BM25:')
for i, text in enumerate(texts[:20]):
    content = json.loads(text.text)['contents']
    print(f'{i+1:2} {text.metadata["docid"]:15} {text.score:.5f} {content}')

BM25:
 1 4348282         10.06630 Civil Records Definition. Civil records are a group of public records that pertain to civil registry records, civil family matters and non criminal civil offenses. These records vary a lot because of the nature of the information that is recorded.
 2 2674124         9.86550 See our FAQ's to learn how you can benefit by using our search. Share: DISCLAIMER: Due to the nature of the origin of public record information, the public records and commercially available data sources used in the PeopleWise system may contain errors.
 3 7119957         9.64430 Yes, in most cases Public Records are available to the public. Some documents, such as certain court records, confidential personal information, and other sensitive information may be kept sealed or is only available with a court order. In certain states, there is a waiting period to obtain Public Records that reveal private information. Which government agencies provide Public Records? A number of governme

In [5]:
!nvidia-smi

Tue Nov  2 00:27:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from pygaggle.rerank.transformer import MonoT5

# This step loads the weights of model `castorini/monot5-base-msmarco`, available at HuggingFace's model hub.
reranker = MonoT5() 

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

In [7]:
reranked = reranker.rerank(query, texts)

print('Reranked:')
# Print out reranked results:
for i, text in enumerate(reranked[:20]):
    content = json.loads(text.text)['contents']
    print(f'{i+1:2} {text.metadata["docid"]:15} {text.score:.5f} {content}')

Reranked:
 1 2647769         -0.00595 Naturalization Record Search The Naturalization and Petition of Record books are available in the Prothonotary Office for public viewing.
 2 8010561         -0.01348 Department of State >> Delaware Public Archives. Naturalization Records Database. Naturalization records can be found in a number of series amongst the holdings of the Delaware Public Archives. The earliest material can be found in the records of the Court of Common Pleas and the Supreme Court of each county.
 3 8010558         -0.02736 Pre-1906 naturalization records (whether they are certificates of naturalizations, declarations of intention, petitions, or oaths and whether the naturalization was granted in state or federal court) provide far less information about the applicant than do the post-1906 records.
 4 8059826         -0.12631 Are all types of Public Records open to the public? Yes, in most cases Public Records are available to the public. Some documents, such as certain co